# Predict Complaint Types

The goal of this exercise is to do Model Development and Validation to find the answer to the Question 4 of the problem statement:

> Can a predictive model be built for future prediction of the possibility of complaints of the specific type that you identified in response to Question 1?

This exercise will be based on the findings of the previous three exercises. Therefore, we shall use the 311 complaints and the PLUTO data sets to feature-engineer a 'HEAT/HOT WATER' complaints for tax lots dataset. The latter is to be used to build a predictive model to estimate the number of future complaints based on selected house characteristics (which we will also refer to as properties or features).

We shall formalize the question at hand as follows:

> Build a prediction model for the number of 'HEAT/HOT WATER' complaints per year for a house with a selectd set of characteristics.

The rest of the work will be organized as follows, we shall:
1. Load, clean and prepare the data sets
   * In a similar way we did for answering Questions 1 to 3
2. Join the '311 complaint' data set with the PLUTO one
   * In a similar way we did for answering Questions 1 to 3
3. Determine the models to be used
   * This will influence the feature selection process
4. Perform house feature selection
   * This will be re-done as we used a different model for Question 3
5. Perform model training
   * Including parameter tuning and cross validation if any
6. Evaluate and compare models
7. Recommending the best performing model

Please note that, the data sets will not be described as the latter has already been done when answering Questions 1 to 3. We shall only repeat that the PLUTO data set will initially be taked with the following set of features:

In [1]:
pluto_features = ['Address', 'BldgArea', 'BldgDepth', 'BuiltFAR',
              'CommFAR', 'FacilFAR', 'Lot', 'LotArea', 'LotDepth',
              'NumBldgs', 'NumFloors', 'OfficeArea', 'ResArea',
              'ResidFAR', 'RetailArea', 'YearBuilt', 'YearAlter1',
              'ZipCode', 'YCoord', 'XCoord']
print('The initial set of PLUTO features to consider:\n', pluto_features)

The initial set of PLUTO features to consider:
 ['Address', 'BldgArea', 'BldgDepth', 'BuiltFAR', 'CommFAR', 'FacilFAR', 'Lot', 'LotArea', 'LotDepth', 'NumBldgs', 'NumFloors', 'OfficeArea', 'ResArea', 'ResidFAR', 'RetailArea', 'YearBuilt', 'YearAlter1', 'ZipCode', 'YCoord', 'XCoord']


# 1. Load, clean, prepare

Loading of the data can be done both from the IBM cloud storage and the locally present CSV files. The latter is decided upon the presence of the proper secure field values of the credentials:

In [2]:
import os
import re
import seaborn
import ibm_boto3
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from botocore.client import Config
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

%matplotlib inline

In [3]:
# @hidden_cell
SECURITY_DUMMY = '----------------'
erm2_nwe9_creds = {
    'IAM_SERVICE_ID'    : SECURITY_DUMMY,
    'IBM_API_KEY_ID'    : SECURITY_DUMMY,
    'ENDPOINT'          : 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT' : 'https://iam.eu-gb.bluemix.net/oidc/token',
    'BUCKET'            : SECURITY_DUMMY,
    'FILE'              : 'erm2_nwe9.csv'
}
bk_18v1_creds = {
    'IAM_SERVICE_ID'    : SECURITY_DUMMY,
    'IBM_API_KEY_ID'    : SECURITY_DUMMY,
    'ENDPOINT'          : 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT' : 'https://iam.eu-gb.bluemix.net/oidc/token',
    'BUCKET'            : SECURITY_DUMMY,
    'FILE'              : 'BK_18v1.csv'
}
bx_18v1_creds = {
    'IAM_SERVICE_ID'    : SECURITY_DUMMY,
    'IBM_API_KEY_ID'    : SECURITY_DUMMY,
    'ENDPOINT'          : 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT' : 'https://iam.eu-gb.bluemix.net/oidc/token',
    'BUCKET'            : SECURITY_DUMMY,
    'FILE'              : 'BX_18v1.csv'
}
mn_18v1_creds = {
    'IAM_SERVICE_ID'    : SECURITY_DUMMY,
    'IBM_API_KEY_ID'    : SECURITY_DUMMY,
    'ENDPOINT'          : 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT' : 'https://iam.eu-gb.bluemix.net/oidc/token',
    'BUCKET'            : SECURITY_DUMMY,
    'FILE'              : 'MN_18v1.csv'
}
qn_18v1_creds = {
    'IAM_SERVICE_ID'    : SECURITY_DUMMY,
    'IBM_API_KEY_ID'    : SECURITY_DUMMY,
    'ENDPOINT'          : 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT' : 'https://iam.eu-gb.bluemix.net/oidc/token',
    'BUCKET'            : SECURITY_DUMMY,
    'FILE'              : 'QN_18v1.csv'
}
si_18v1_creds = {
    'IAM_SERVICE_ID'    : SECURITY_DUMMY,
    'IBM_API_KEY_ID'    : SECURITY_DUMMY,
    'ENDPOINT'          : 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT' : 'https://iam.eu-gb.bluemix.net/oidc/token',
    'BUCKET'            : SECURITY_DUMMY,
    'FILE'              : 'SI_18v1.csv'
}

pluto_creds = [bk_18v1_creds, bx_18v1_creds, mn_18v1_creds, qn_18v1_creds, si_18v1_creds]

In [4]:
# Allows to get the data source for the credentials from the IBM cloud or local csv file 
def get_data_source(credentials) :
    '''Creates a data source from the IBM cloud or local csv file according to the credentials'''
    # Here we check if the credentials are present, if not try 
    # load the local file if they are then read from the cloud.
    if credentials.get('IAM_SERVICE_ID') == SECURITY_DUMMY :
        # This is the alternative to get the code run locally with a local csv file
        body = 'data' + os.path.sep + credentials.get('FILE')
    else :
        client = ibm_boto3.client(
            service_name = 's3',
            ibm_api_key_id = credentials.get('IBM_API_KEY_ID'),
            ibm_auth_endpoint = credentials.get('IBM_AUTH_ENDPOINT'),
            config = Config(signature_version='oauth'),
            endpoint_url = credentials.get('ENDPOINT'))

        body = client.get_object(
            Bucket = credentials.get('BUCKET'),
            Key = credentials.get('FILE'))['Body']

        # add missing __iter__ method, so pandas accepts body as file-like object
        def __iter__(self): return 0
        if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

    return body

Further, we shall subsequently load the 311 and PLUTO data sets. Along the way, we will select the necessary columns and check on (, and correct if needed,) the column data types.

## 311 complaints

Balow we load the data set, and then first select the required complaints along with the needed columns:

In [5]:
# Get the data source for the credentials
dhp_ds = get_data_source(erm2_nwe9_creds)

# Read the CSV file
dhp_df = pd.read_csv(dhp_ds, parse_dates = ['created_date', 'closed_date'])
print('Number of all complaints:', dhp_df.shape[0])

Number of all complaints: 6034470


In [6]:
# Select the 'HEAT/HOT WATER' complaints
dhp_df = dhp_df[(dhp_df['complaint_type'] == 'HEAT/HOT WATER')]
print('Number of \'HEAT/HOT WATER\' complaints is:', dhp_df.shape[0])

Number of 'HEAT/HOT WATER' complaints is: 2159103


In [7]:
# Select the columns that matter, and rename for convenience
dhp_df = dhp_df[['created_date', 'incident_address', 'incident_zip']]
dhp_df = dhp_df.rename({'incident_address':'Address', 'incident_zip':'ZipCode'}, axis=1)

In [8]:
# Convert the address to upper case for uniformity
dhp_df.Address = dhp_df.Address.map(str).map(str.upper)
# Strip the address strings
dhp_df.Address = dhp_df.Address.str.strip()
# Replace sequence of white spaces with one
dhp_df.Address = dhp_df.Address.str.replace('\s+',' ')

In [9]:
# Drop the Na/NaN valued rows
init_size = dhp_df.shape[0]
dhp_df.dropna(inplace = True)
print('Number of rows before dropping Na/NaN:', init_size,', after:', dhp_df.shape[0])

Number of rows before dropping Na/NaN: 2159103 , after: 2140078


Exract the year the complaint was created and then drop the *'created_date'* column.

In [10]:
dhp_df['Year'] = dhp_df.created_date.dt.year
dhp_df.drop(columns = ['created_date'], inplace = True)

Let us summarize the Year statitics so far:

In [11]:
year_descr = dhp_df['Year'].describe().astype(int)
year_descr

count    2140078
mean        2014
std            2
min         2010
25%         2012
50%         2015
75%         2017
max         2020
Name: Year, dtype: int64

As one can see the min/max years range is between `2010` and `2020` which means that there were no missing/wrong *'created_date'* values present.

We will only need the average complaint counts per year for each given *'Address'*/*'ZipCode'* pair:

In [12]:
# Group by the zip code and address to count the complaints
dhp_df = dhp_df.groupby(['ZipCode', 'Address']).size().to_frame()

# Rename the counts column and then compute the average count for the min/max years range
dhp_df.rename({0 : 'AvgCnt'}, axis = 1, inplace = True)
dhp_df.AvgCnt = dhp_df.AvgCnt/(year_descr.loc['max'] - year_descr.loc['min'] + 1)

# Re-set the indexes to turn the Address and ZipCode back into columns
dhp_df.reset_index(level = 1, inplace = True)
dhp_df.reset_index(level = 0, inplace = True)
dhp_df.head()

,ZipCode,Address,AvgCnt
0,10001.0,10 WEST 28 STREET,1.454545
1,10001.0,100 WEST 26 STREET,0.090909
2,10001.0,102 WEST 29 STREET,0.727273
3,10001.0,103 WEST 27 STREET,0.181818
4,10001.0,11 WEST 34 STREET,0.090909


Finally, we check on the column types:

In [13]:
dhp_df.dtypes

ZipCode    float64
Address     object
AvgCnt     float64
dtype: object

All the data types are in order: the address is a string and the zip code and average complaint count are floats.

## PLUTO 

Let us load and combine all of the PLUTO csv files first:

In [14]:
# Iterate to load all the PLUTO csv files and
# re-combine them into a single data frame
pluto_df = pd.DataFrame()
for cred in pluto_creds :
    # Get the data source for the credentials
    ds = get_data_source(cred)
    # Read and append the CSV file
    pluto_df = pluto_df.append(pd.read_csv(ds, low_memory = False))

Perform initial pre-processing such as, selecting the right rows, dropping the Na/NaN valued rows, pre-processing the *'Address'* column values to match those of the '311 complaints' data set

In [15]:
num_pluto_rows = pluto_df.shape[0]

# Select the recommended columns
pluto_df = pluto_df[['Address', 'BldgArea', 'BldgDepth', 'BuiltFAR',
              'CommFAR', 'FacilFAR', 'Lot', 'LotArea', 'LotDepth',
              'NumBldgs', 'NumFloors', 'OfficeArea', 'ResArea',
              'ResidFAR', 'RetailArea', 'YearBuilt', 'YearAlter1',
              'ZipCode', 'YCoord', 'XCoord']]

# Drop the Na/NaN valued rows
pluto_df = pluto_df.dropna()

# Convert the address to upper case for uniformity
pluto_df.Address = pluto_df.Address.map(str).map(str.upper)
# Strip the address strings
pluto_df.Address = pluto_df.Address.str.strip()
# Replace sequence of white spaces with one
pluto_df.Address = pluto_df.Address.str.replace('\s+',' ')

print('The number of PLUTO rows before:', num_pluto_rows, ', after:', pluto_df.shape[0])

The number of PLUTO rows before: 859212 , after: 835242


As we know from aanswering Question 3, there are Address column values in the PLUTO data set that do not contain house numbers. As, before, let us drop entries those right away:

In [16]:
num_pluto_rows = pluto_df.shape[0]

# Create the pattern for selecting a proper address with one house
# number followed by a white space and then an alpha numeric street
# name which can contain multiple words separated by white spaces
regex_pat = re.compile(r'^\d+(\s+\w+)+')

# Then we only select the addresses that have a house number in it
pluto_df = pluto_df[pluto_df.Address.str.match(regex_pat)]

print('The number of PLUTO rows before:', num_pluto_rows, ', after:', pluto_df.shape[0])

The number of PLUTO rows before: 835242 , after: 522280


Next, the same as done for Question 3, we shall remove the entries with the duplicate *'Address'*/*'Zipcode'* pairs:

In [17]:
num_pluto_rows = pluto_df.shape[0]

# First count the distinct ZipCode-Address groups
occur_cnt = pluto_df.groupby(['ZipCode','Address']).size().to_frame()
occur_cnt.reset_index(level=0, inplace = True)
occur_cnt.reset_index(level=0, inplace = True)

# Rename the counts column
occur_cnt.rename(columns={0:'Count'}, inplace = True)

# Join the data into the original table
pluto_df_tmp = pd.merge(pluto_df, occur_cnt, left_on=['ZipCode','Address'], right_on=['ZipCode','Address'])

# Only keep the Address - ZipCode pairs that occur once
rows_index = pluto_df_tmp[pluto_df_tmp['Count'] > 1].index
pluto_df = pluto_df_tmp.drop(rows_index, axis = 0)

# Drop the counts columns is it will not be needed and remove the temporary object
pluto_df = pluto_df.drop('Count', axis = 1)
pluto_df_tmp = []

print('The number of PLUTO rows before:', num_pluto_rows, ', after:', pluto_df.shape[0])

The number of PLUTO rows before: 522280 , after: 520137


Finally, let us check on the column data types, as we see there is no unexpected types here:

In [18]:
pluto_df.dtypes

Address        object
BldgArea        int64
BldgDepth     float64
BuiltFAR      float64
CommFAR       float64
FacilFAR      float64
Lot             int64
LotArea         int64
LotDepth      float64
NumBldgs        int64
NumFloors     float64
OfficeArea      int64
ResArea         int64
ResidFAR      float64
RetailArea      int64
YearBuilt       int64
YearAlter1      int64
ZipCode       float64
YCoord        float64
XCoord        float64
dtype: object

# 2. Join data sets

In this section we shall join the cleaned 311 and PLUTO data sets to form a single data frame, we shall do it by performing an inner joint on the 311 and PLUTO data frames. The goint will be done by *'Address'* and *'ZipCode'* columns. This way we will only get the complaints that are marched with the PLUTO tax lot. This approach solves the three yet unresolved issues with the PLUTO data set mentioned in the previous section by ignoring the complaints that could not be matched with the tax lots because of the missing house numbers.

In [19]:
num_311_rows = dhp_df.shape[0]

dhp_pluto_df = pd.merge(dhp_df, pluto_df, how='inner', left_on=['ZipCode','Address'], right_on=['ZipCode','Address'])

print('The number of 311 rows before:', num_311_rows, ', after:', dhp_pluto_df.shape[0])

The number of 311 rows before: 126902 , after: 82937


As one can see the inner joint has reduced our data quite significantly, i.e. by about `100 - (82937 * 100) / 126902 = 34.64`%. However, this is the price we have to pay unless we do a more thorough pre-porocessing of the PLUTO data set. 

Please note that, the number of house properties (features) that is selected is currently rather large:

In [20]:
NUM_FEATURES = (dhp_pluto_df.shape[1] - len(['Lot', 'Address', 'ZipCode', 'AvgCnt']))
print('The number of selected features is:', NUM_FEATURES)

The number of selected features is: 17


We do not consider *'Lot'*, *'Address'*, *'ZipCode'*, and *'AvgCnt'* as the former two were used to join the 311 and PLUTO data sets and are not directly related to house properties. The latter is the target variable, i.e. the one that shall depend on the house properties.

It does not seem that the *'Lot'*, *'Address'* and *'ZipCode'* columns will be needed so let us drop them:

In [21]:
dhp_pluto_df.drop(columns = ['Lot', 'Address', 'ZipCode'], inplace = True)
display(dhp_pluto_df.head())

,AvgCnt,BldgArea,BldgDepth,BuiltFAR,CommFAR,FacilFAR,LotArea,LotDepth,NumBldgs,NumFloors,OfficeArea,ResArea,ResidFAR,RetailArea,YearBuilt,YearAlter1,YCoord,XCoord
0,1.454545,9763,98.0,3.95,10.0,10.0,2469,98.75,1,5.0,0,4875,0.0,4888,1930,1979,210644.0,987501.0
1,0.727273,8970,82.0,4.16,10.0,10.0,2156,98.75,1,5.0,0,0,10.0,1790,1920,1989,211297.0,986855.0
2,0.181818,4796,76.0,2.88,10.0,10.0,1668,83.42,1,4.0,0,3276,10.0,1520,1920,1989,210983.0,986664.0
3,0.090909,17153,120.0,5.42,15.0,15.0,3162,126.50,1,6.0,1877,0,10.0,7765,1920,0,212224.0,988263.0
4,0.545455,20422,96.0,4.83,10.0,10.0,4229,98.75,2,7.0,0,12866,0.0,4834,1920,2005,211262.0,986709.0


Finally, let us double-check that there are no Na/NaN values present in the resulting joint data frame:

In [22]:
dhp_pluto_df.isna().describe()

,AvgCnt,BldgArea,BldgDepth,BuiltFAR,CommFAR,FacilFAR,LotArea,LotDepth,NumBldgs,NumFloors,OfficeArea,ResArea,ResidFAR,RetailArea,YearBuilt,YearAlter1,YCoord,XCoord
count,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937
unique,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
freq,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937,82937


According to the table above there are no Na/NaN values present so we can proceed with building models to predict the number of 'HEAT/HOT WATER' comlaints per year based on house properties.

# 3. Model selection

# 4. Feature selection

## 4.1 Univariate Selection

## 4.2 Feature Importance

## 4.3 Feature correlations

## 4.4 Final selection

# 5. Split the data

# 6. Train models

# 7. Evaluate models

## 7.1 Model 1

## 7.2 Model 2

## 7.3 Evaluation summary

# 8. Conclusions